## PD 6, dane FICO
### Tomek Radzikowski

### Wybór modelu, wybór zmiennych
### Model XG-Boost

Zbiór danych zawiera zmienną ExternalRiskEstimate (odpowiednik ratingu polskiego BIKu), która w pewien sposób agreguje inne cechy zawarte i niezawarte w zbiorze. Związku z powyższym stawiam hipotezę, że jej obecność może być kłopotliwa dla wyjaśnień (jej obecność może sprawić, że inne cechy będą zmarginalizowane). Z poprzednich zmagać z tym zbiorem wiem też, że wyrugowanie ExternalRiskEstimate nie powoduje poważnego spadku jakości predykcji, więc można sobie na to pozwolić. Celem potwierdzenia hipotezy wykonałem wykresy PDP dla obu wersji modelu (posiadającej ExternalRisk i nieposiadającej). Efekty zdają się potwierdzać założenie: dla większości predyktorów zmienność prawdopodobieństwa, jaką one zapewniają jest większa, w przypadku usunięcia ExternalRiskEstimate, czyli waga tych zmiennych nie jest tak "spłaszczona" - zmarginalizowana. Poza tym, intuicyjnie, niektóre wykresy PDP mają zgodny z oczekiwaniami przegląd. Na przykład zmienna MSinceMostRecentDelq oznaczająca liczbę miesięcy od ostatniego opóźnienia w spłacie jest funkcją niemalejącą dla modelu bez zmiennej ExternalRisk, w przeciwnym przypadku tak nie jest. Można przyjąć, że zgodnie ze zdrowym rozsądkiem lepiej jest, gdy od ostatniego opóźnienia spłaty kredytu minęło więcej czasu, co nie jest spełnione w modelu z ExternalRisk. Na tej podstawie do dalszych analiz wybieram model bez zmiennej ExternalRiskEstimate.
<img src="images/h69.png">
<img src="images/h69a.png">

### Model NN
Drugi wytrenowany model - sieć neuronowa również został sprawdzony dla wersji z ExternalRiskEstimate i bez. Ponownie wybrano wersję bez tej zmiennej.


### ALE vs PDP dla jednego modelu

Praktycznie dla wszystkich zmiennych istnieje znacząca różnica w przebiegu PDP i ALE co oznacza, że zmienne nie są od sibie niezależne. Poniżej wykresy dla modelu XGBoost.

<img src="images/h61.png">
<img src="images/ha61.png">


### Porównanie wyników dla obu modeli.

Pierwszą, rzucającą się w oczy różnicą, która wynika wprost z budowy modeli jest przebieg funkcji dla poszczególnych zmiennych. Dla algorytmów drzewiastych przebiegi składają się z łamanych, natomiast dla sieci neuronowych są gładkie, pozbawione wartości stałych. 

### PDP

Do porównania wybrałem wykresy następujących zmiennych:
MsinceOldestTradeOpen, AvarageMInFile, MaxDelqPublicL12M, PercentTradesWBalance

Górny wykres zawsze odnosi się do modelu sieci neuronowej, dolny do XGBoost.
Można zauważyć, że zmienne dzielą się na dwie grupy: jedna - wykresy dla obu modeli są podobne, obrazują tę samą tendencję, dla sieci neuronowej widać tylko wygładzenie przebiegu:

MsinceOldestTradeOpen
<img src="images/hn1.png">
<img src="images/h62.png">

AvarageMInFile
<img src="images/hn2.png">
<img src="images/h64.png">

Oraz druga, gdzie przebiegi różnią się diametralnie.

MaxDelqPublicL12M
<img src="images/hn4.png">
<img src="images/h610.png">


PercentTradesWBalance
<img src="images/hn7.png">
<img src="images/h623.png">

### ALE

Górny wykres zawsze odnosi się do modelu sieci neuronowej, dolny do XGBoost.
Dla pewnych zmiennych wykresy ALE obu modeli są podobne:

MsinceOldestTradeOpen:
<img src="images/han1.png">
<img src="images/ha61.png">

PercentInstallTrades:
<img src="images/han8.png">
<img src="images/ha613.png">



W poniższych przypadkach wydaje się, że lepiej odwzorowują stan faktyczny wykresy dla modelu sieci neuronowej. Widać to poniżej.


NumTrades90Ever2DerogPubRec:
Wykres ALE tej zmiennej jest moim zdaniem lepszy dla sieci neuronowej, ponieważ nie jest stały, tj. ma wpływ na pozytywną decyzję kredytową.

<img src="images/han3.png">
<img src="images/ha66.png">

W dwóch poniższych przypadkach widać, że w modelu sieci neuronowej przebiegi są "łaskawsze" dla potencjalnych kredytobiorców, to znaczy wartość funkcji zmienia się wolniej, ale na dłuższym odcinku. Model XGBoost miał tendencję do zwiększenia wartości funkcji w pobliżu skrajnych wartości zmiennej.

PercentTradesNeverDelq:
<img src="images/han4.png">
<img src="images/ha67.png">

MaxDelq2PublicRecLast12M:
<img src="images/han5.png">
<img src="images/ha69.png">



Niektóre wyniki są zupełnie niezgodne między modelami. Jest to prawdopodobnie związane z zupełnie różną specyfiką budowy obu modeli, w związku z czym ocena niektórych predyktorów jest wręcz antagoniczna.

NumTotalTrades:
<img src="images/han7.png">
<img src="images/ha611.png">



In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import xgboost as xgb
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import explained_variance_score
import matplotlib.pyplot as plt
import re

In [32]:
data=pd.read_csv("heloc_dataset_v1.csv")
data.head()

print('braki: ',data.isnull().values.any())


data.RiskPerformance=data.RiskPerformance.replace(to_replace=['Bad', 'Good'], value=[0, 1])
data['RiskPerformance'] = pd.to_numeric(data['RiskPerformance']) 

data.dtypes

braki:  False


RiskPerformance                       int64
ExternalRiskEstimate                  int64
MSinceOldestTradeOpen                 int64
MSinceMostRecentTradeOpen             int64
AverageMInFile                        int64
NumSatisfactoryTrades                 int64
NumTrades60Ever2DerogPubRec           int64
NumTrades90Ever2DerogPubRec           int64
PercentTradesNeverDelq                int64
MSinceMostRecentDelq                  int64
MaxDelq2PublicRecLast12M              int64
MaxDelqEver                           int64
NumTotalTrades                        int64
NumTradesOpeninLast12M                int64
PercentInstallTrades                  int64
MSinceMostRecentInqexcl7days          int64
NumInqLast6M                          int64
NumInqLast6Mexcl7days                 int64
NetFractionRevolvingBurden            int64
NetFractionInstallBurden              int64
NumRevolvingTradesWBalance            int64
NumInstallTradesWBalance              int64
NumBank2NatlTradesWHighUtilizati

In [33]:
y=data.pop('RiskPerformance')
X=data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=421)



In [34]:
X_train.head()

,ExternalRiskEstimate,MSinceOldestTradeOpen,MSinceMostRecentTradeOpen,AverageMInFile,NumSatisfactoryTrades,NumTrades60Ever2DerogPubRec,NumTrades90Ever2DerogPubRec,PercentTradesNeverDelq,MSinceMostRecentDelq,MaxDelq2PublicRecLast12M,...,PercentInstallTrades,MSinceMostRecentInqexcl7days,NumInqLast6M,NumInqLast6Mexcl7days,NetFractionRevolvingBurden,NetFractionInstallBurden,NumRevolvingTradesWBalance,NumInstallTradesWBalance,NumBank2NatlTradesWHighUtilization,PercentTradesWBalance
9398,59,213,24,90,13,0,0,85,5,4,...,54,1,10,10,104,-8,2,1,2,100
5945,74,77,1,35,15,0,0,100,-7,7,...,50,-7,0,0,47,72,4,3,1,88
7087,63,211,13,76,15,2,2,74,9,2,...,5,0,7,7,1,-8,1,-8,0,14
4113,92,132,15,103,24,0,0,100,-7,7,...,52,4,1,1,0,-8,1,-8,0,50
178,66,93,7,33,23,0,0,100,-7,7,...,33,0,2,2,58,97,8,2,4,63


In [5]:
model = xgb.XGBClassifier()

train_model=model.fit(X_train, y_train)

In [6]:
from sklearn.metrics import accuracy_score
pred = train_model.predict(X_test)
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred) * 100))

Accuracy for model 1: 73.06


In [ ]:
from sklearn.inspection import plot_partial_dependence
for i in X_test:

    disp1 = plot_partial_dependence(train_model, X_test, [i])  

In [37]:
model1 = xgb.XGBClassifier()
X_train=X_train.drop('ExternalRiskEstimate', 1)
X_test=X_test.drop('ExternalRiskEstimate', 1)
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=421)
train_model1=model1.fit(X_train, y_train)
pred = train_model1.predict(X_test)
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, pred) * 100))

Accuracy for model 1: 73.09


In [ ]:
for i in X_test:

    disp1 = plot_partial_dependence(train_model1, X_test, [i])  

In [27]:
from alepython import ale_plot

In [ ]:
for i in X_test:

    disp1 = ale_plot(train_model1, X_test, i, monte_carlo=True)

In [39]:
from sklearn.neural_network import MLPClassifier

In [41]:
model_nn = MLPClassifier(max_iter=500, activation='relu', batch_size=64)

train_model_nn=model_nn.fit(X_train, y_train)
print("Accuracy for model 1: %.2f" % (accuracy_score(y_test, model_nn.predict(X_test)) * 100))

Accuracy for model 1: 70.28


In [ ]:
for i in X_test:

    disp1 = plot_partial_dependence(train_model_nn, X_test, [i]) 

In [ ]:
for i in X_test:
    disp1 = ale_plot(train_model_nn, X_test, i, monte_carlo=True)